<a href="https://colab.research.google.com/github/hpputro/ml4sc/blob/main/3_Permodelan_Bahasa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Persiapan

In [ ]:
!pip install antlr4-tools
!pip install antlr4-python3-runtime==4.11.1
!pip install pddlpy
!antlr4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for install-jdk: filename=install_jdk-0.3.0-py3-none-any.whl size=3740 sha256=7cfc835dd1d33ab2325190156027797b0f062bd105405c78a470de565d971f79
  Stored in directory: /root/.cache/pip/wheels/89/a9/a3/03dc102cdcd442b9bca361f8c64fd4bb9b47ce75d9c8d56c91
Successfully built install-jdk
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 KB 8.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 KB 5.5 MB/s eta 0:00:00
ANTLR Parser Generator  Version 4.11.1
 -o ___              specify output directory where all output is generated
 -lib ___            specify location of grammars, tokens files
 -atn                gene

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
!dir
!cp "drive/MyDrive/Colab Notebooks/SourceCode/arithmeticLexer.py" .
!cp "drive/MyDrive/Colab Notebooks/SourceCode/arithmeticParser.py" .

drive  sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Baca Source Code

In [ ]:
import sys
import pandas as pd
from google.colab import drive

drive.mount('/drive')
df = pd.read_csv('/drive/My Drive/Colab Notebooks/SourceCode/sourcecode2.csv')
X = df['source']
y = df['label']
print(X.head())
print(y.head())

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
0                       java = 123 * 4567
1         private int mainan(String args)
2    // two integer variables with values
3                            int num1 = 5
4                           int num2 = 15
Name: source, dtype: object
0     expression
1    declaration
2        comment
3    declaration
4    declaration
Name: label, dtype: object


# Ubah ke Token Stream

In [ ]:
from antlr4 import *
from arithmeticLexer import arithmeticLexer
from arithmeticParser import arithmeticParser

file = []
for line, label in zip(X,y):
  f = open("/drive/My Drive/Colab Notebooks/SourceCode/test3.txt", "w")
  f.write(line)
  f.close()

  input_stream = FileStream("/drive/My Drive/Colab Notebooks/SourceCode/test3.txt")
  lexer = arithmeticLexer(input_stream)
  tokens = lexer.getAllTokens()
  line = ''
  for tk in tokens:
    name = lexer.symbolicNames[tk.type]
    line = line + ' ' + name
  file.append([line,label])
print(file)

df2 = pd.DataFrame(file,columns=['source','label'])
df2.to_csv('/drive/My Drive/Colab Notebooks/SourceCode/tokenstream2.csv')


[[' VARIABLE EQ SCIENTIFIC_NUMBER TIMES SCIENTIFIC_NUMBER', 'expression'], [' VARIABLE VARIABLE VARIABLE LPAREN VARIABLE VARIABLE RPAREN', 'declaration'], [' DIV DIV VARIABLE VARIABLE VARIABLE VARIABLE VARIABLE', 'comment'], [' VARIABLE VARIABLE EQ SCIENTIFIC_NUMBER', 'declaration'], [' VARIABLE VARIABLE EQ SCIENTIFIC_NUMBER', 'declaration'], [' VARIABLE VARIABLE', 'declaration'], [' DIV DIV VARIABLE VARIABLE VARIABLE VARIABLE VARIABLE VARIABLE VARIABLE VARIABLE', 'comment'], [' VARIABLE EQ VARIABLE', 'expression'], [' DIV DIV VARIABLE VARIABLE VARIABLE', 'comment'], [' VARIABLE POINT VARIABLE POINT VARIABLE LPAREN VARIABLE PLUS VARIABLE PLUS VARIABLE PLUS VARIABLE PLUS VARIABLE PLUS VARIABLE RPAREN', 'expression'], [' VARIABLE VARIABLE POINT VARIABLE POINT VARIABLE', 'declaration'], [' VARIABLE EQ LPAREN SCIENTIFIC_NUMBER PLUS SCIENTIFIC_NUMBER RPAREN TIMES SCIENTIFIC_NUMBER', 'expression'], [' VARIABLE VARIABLE LPAREN VARIABLE VARIABLE RPAREN', 'declaration'], [' VARIABLE VARIABLE EQ

# TF IDF

## Dengan TF IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=3, norm='l2', encoding='latin-1', ngram_range=(1, 2))
ft = tfidf.fit_transform(file)
features = ft.toarray()
print(tfidf.get_feature_names_out())
labels = df.label
print(features.shape)


['div' 'div div' 'div variable' 'eq' 'eq eq' 'eq lparen'
 'eq scientific_number' 'eq variable' 'lparen' 'lparen rparen'
 'lparen scientific_number' 'lparen variable' 'lt' 'lt variable' 'minus'
 'plus' 'plus plus' 'plus rparen' 'plus scientific_number' 'plus variable'
 'point' 'point variable' 'rparen' 'scientific_number'
 'scientific_number rparen' 'scientific_number times' 'times'
 'times variable' 'variable' 'variable eq' 'variable lparen' 'variable lt'
 'variable minus' 'variable plus' 'variable point' 'variable rparen'
 'variable times' 'variable variable']
(96, 38)


## Dengan Count Vectorizer & TF IDF Transformer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df2['source'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf)

  (0, 10)	0.1423281276855254
  (0, 9)	0.48071131576144577
  (0, 8)	0.797641628559877
  (0, 1)	0.3353016065526375
  (1, 10)	0.9087711396736358
  (1, 7)	0.29507203840442203
  (1, 2)	0.29507203840442203
  (2, 10)	0.6572636295029801
  (2, 0)	0.7536607468434118
  (3, 10)	0.4794378367086556
  (3, 8)	0.6717217163680992
  (3, 1)	0.5647382548505274
  (4, 10)	0.4794378367086556
  (4, 8)	0.6717217163680992
  (4, 1)	0.5647382548505274
  (5, 10)	1.0
  (6, 10)	0.812817694200482
  (6, 0)	0.5825181507855456
  (7, 10)	0.6471861817537802
  (7, 1)	0.7623319789612417
  (8, 10)	0.463622721859743
  (8, 0)	0.8860327148448658
  (9, 10)	0.5209820923310365
  (9, 7)	0.09397748554500836
  (9, 6)	0.23052786561260744
  :	:
  (88, 8)	0.47592127632423786
  (88, 7)	0.2757347966119527
  (88, 4)	0.7113738802320405
  (88, 2)	0.2757347966119527
  (89, 10)	0.5465335477275783
  (89, 7)	0.295759773728239
  (89, 6)	0.7255021665693002
  (89, 2)	0.295759773728239
  (90, 10)	1.0
  (91, 10)	0.6805236931518441
  (91, 7)	0.22096158

# Klasifikasi

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, labels)

# Prediksi

## Dari Token Stream

In [ ]:
print(clf.predict(count_vect.transform(["variable plus variable"])))

['expression']


## Dari Baris Source Code

In [ ]:
f = open("/drive/My Drive/Colab Notebooks/SourceCode/test3.txt", "w")
f.write("// ini komentar")
f.close()

input_stream = FileStream("/drive/My Drive/Colab Notebooks/SourceCode/test3.txt")
lexer = arithmeticLexer(input_stream)
tokens = lexer.getAllTokens()
line = ''
for tk in tokens:
  name = lexer.symbolicNames[tk.type]
  line = line + ' ' + name
print(clf.predict(count_vect.transform([line])))

['comment']
